In [5]:
import pandas as pd
df = pd.read_csv("animated_movies.csv")
df.head()

# movie_info['Year'] = info_we[0].find_element(By.CLASS_NAME, 'scoreboard__info').text.split(',')[0]

,Movie Name,Audience Score,Tomatometer Score,Rating,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),Box Office (Gross USA),Runtime,Distributor,Year,Aspect Ratio,Sound Mix,View the collection
0,REDLINE,86.0,70,NaN,"Action, Sci-Fi, Anime",Japanese,Takeshi Koike,NaN,"Katsuhito Ishii, Katsuhito Ishii",NaN,"Jan 17, 2012",NaN,1h 42m,NaN,NaN,NaN,NaN,NaN
1,THE THIEF AND THE COBBLER,74.0,56,G,"Kids & Family, Action, Adventure, Comedy, Fant...",English,Richard Williams,"Jacobus Rose, Fred Calvert","Richard Williams, Margaret French","Aug 25, 1995 Original","May 3, 2011",$635.4K,1h 12m,Miramax Films,NaN,NaN,NaN,NaN
2,MONSTER HOUSE,63.0,75,PG (Scary Images and Sequences|Brief Language|...,"Kids & Family, Fantasy, Adventure, Animation",English,Gil Kenan,"Jack Rapke, Steve Starkey","Ron Schrab, Dan Harmon, Pamela Pettler","Jul 21, 2006 Wide","Oct 24, 2006",$73.7M,1h 31m,Sony Pictures Entertainment,NaN,Scope (2.35:1),"Dolby SRD, DTS, SDDS",NaN
3,ANASTASIA,77.0,86,G,"Kids & Family, Musical, Animation",English,"Don Bluth, Gary Goldman",Don Bluth,"Bruce Graham, Bob Tzudiker, Noni White, Susan ...","Nov 21, 1997 Original","Aug 3, 2004",$58.4M,1h 34m,20th Century Fox,NaN,NaN,Surround,NaN
4,PARANORMAN,72.0,89,PG (Scary Action|Language|Scary Images|Some Ru...,"Kids & Family, Comedy, Fantasy, Adventure, Ani...",English,"Chris Butler, Sam Fell","Arianne Sutner, Travis Knight",Chris Butler,"Aug 17, 2012 Wide","Nov 27, 2012",NaN,1h 36m,Focus Features,NaN,Scope (2.35:1),"DTS, Dolby Atmos, Dolby SR, Dolby Digital, Dol...",NaN


In [ ]:
# Has Audience Score?
# Rating Letter
# Number of Genre
# Number of Directors
# Number of Producers
# Number of Writers
# Release Year
# Stream Year
# Box Office Into Integer
# Runtime (Minutes)
# Remove View the Collection

In [6]:
df.isnull().any()

Movie Name                  False
Audience Score               True
Tomatometer Score           False
Rating                       True
Genre                        True
Original Language            True
Director                    False
Producer                     True
Writer                       True
Release Date (Theaters)      True
Release Date (Streaming)     True
Box Office (Gross USA)       True
Runtime                      True
Distributor                  True
Year                         True
Aspect Ratio                 True
Sound Mix                    True
View the collection          True
dtype: bool

In [7]:
df[df['Original Language'].isnull()]

,Movie Name,Audience Score,Tomatometer Score,Rating,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),Box Office (Gross USA),Runtime,Distributor,Year,Aspect Ratio,Sound Mix,View the collection
42,DAFT PUNK & LEIJI MATSUMOTO'S INTERSTELLA 5555...,95.0,86,NaN,NaN,NaN,Leiji Matsumoto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,WALLACE & GROMIT: THE CURSE OF THE WERE-RABBIT,79.0,95,NaN,NaN,NaN,Steve Box,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
